In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet import ResNet152

from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B1
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet201, DenseNet121
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.regnet import RegNetY320

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, LSTM
import pickle



C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Dataset Preparation

In [9]:
img_size = 128
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "testdata",
    shuffle = True,
    image_size = (img_size, img_size),
    batch_size = 32
)

Found 174 files belonging to 3 classes.


In [10]:
def data_extractor(dataset):
    test_img = []
    img_label = []
    for img, labels in dataset:
        for i in img:
            test_img.append(np.array(i))
        dataset1 = tf.data.Dataset.from_tensor_slices(labels)
        list_data = list(dataset1.as_numpy_iterator())
        for j in list_data:
            img_label.append(j)
    output_class = tf.constant(img_label)

    test_img1 = np.array(test_img)
    output_class1 = np.array(output_class)

    return test_img1, output_class1

# X_train, y_train = data_extractor(train_ds)
X_test, y_test = data_extractor(dataset)
# # X_val, y_val = data_extractor(val_ds)
X_test.shape, y_test.shape

((174, 128, 128, 3), (174,))

In [12]:
pickle.dump(X_test, open("Xtest.pkl", "wb"))
pickle.dump(y_test, open("ytest.pkl", "wb"))

X = pickle.load(open("X_train.pkl", "rb"))
y = pickle.load(open("y_train.pkl", "rb"))

X_test = pickle.load(open("X_test.pkl", "rb"))
y_test = pickle.load(open("y_test.pkl", "rb"))

X_val = pickle.load(open("X_val.pkl", "rb"))
y_val = pickle.load(open("y_val.pkl", "rb"))

## Pretrained Architecture

In [36]:
def run_model(train_index, test_index):
    img_size = 128
    model = Sequential()
    
    model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet', input_shape = (img_size, img_size, 3)))

    model.add(Dense(3, activation='softmax'))

    model.layers[0].trainable=False
    model.build()
    model.summary()

    model.compile(
        optimizer='adam',
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics = ['accuracy']
        )
    
    model_filepath = '/saved_models/weights_improvement-{epoch:02d}-{val_loss:.4f}.hdf5'
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=model_filepath,
        save_weights_only=True,
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
)

    history = model.fit(
        X_test[train_index],
        y_test[train_index],
        epochs = 3,
        batch_size = 32,
        verbose = 1,
        validation_data= (X_val, y_val),
        callbacks=[checkpoint_callback]
    )
    
    folder_path = 'saved_models'
    files = os.listdir(folder_path)
    files.sort()  # Sort files by name
    latest_file = os.path.join(folder_path, files[-1])
    print(latest_file)
    
    model.load_weights(lastest_file)
    
    test_score = model.evaluate(X[test_index], y[test_index])
    predictions = model.predict(X[test_index])
    predicted_classes = np.argmax(predictions, axis=1)
    precision1, recall1, f11, _ = precision_recall_fscore_support(y[test_index], predicted_classes, average='weighted')
    
    return (test_score, precision1, recall1, f11)

In [37]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
img_size = 128

accuracy = []
precision = []
recall = []
f1 = []

for train_index, test_index in kf.split(X):
    score, precision2, recall2, f12 = run_model(train_index, test_index)
    accuracy.append(score[1])
    precision.append(precision2)
    recall.append(recall2)
    f1.append(f12)

print("\n5-fold CV performance parameters")
print("----------------------------------------------------")
print("CV scores : ", accuracy)
print("Avg accuracy : ", np.mean(accuracy))
print("Avg precision : ", np.mean(precision))
print("Avg recall : ", np.mean(recall))
print("Avg f1_score : ", np.mean(f1))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_1 (Dense)             (None, 3)                 6147      
                                                                 
Total params: 23593859 (90.00 MB)
Trainable params: 6147 (24.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________
Epoch 1/3
107/107 [==============================] - ETA: 0s - loss: 0.7423 - accuracy: 0.7085
Epoch 1: val_loss improved from inf to 0.42576, saving model to /saved_models\weights_improvement-01-0.4258.hdf5
107/107 [==============================] - 141s 1s/step - loss: 0.7423 - accuracy: 0.7085 - val_loss: 0.4258 - val_accuracy: 0.8406
Epoch 2/3
107/107 [==============================] - ETA: 0s - los

NameError: name 'lastest_file' is not defined

## Modified Architecture

In [48]:
img_size = 128
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet', input_shape = (img_size, img_size, 3)))

model.add(Dense(256, activation='silu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='silu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='silu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='silu'))

model.add(Dense(3, activation='softmax'))

model.layers[0].trainable=False
model.build()
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_3 (Dense)             (None, 3)                 6147      
                                                                 
Total params: 23593859 (90.00 MB)
Trainable params: 6147 (24.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [56]:
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy']
    )

model_filepath = 'saved_models/weights_improvement-{epoch:02d}-{val_loss:.4f}.hdf5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

history = model.fit(
    X_test,
    y_test,
    epochs = 3,
    batch_size = 32,
    verbose = 1,
    validation_data= (X_val, y_val),
    callbacks=[checkpoint_callback]
)

folder_path = 'saved_models'
files = os.listdir(folder_path)
files.sort()
latest_file = os.path.join(folder_path, files[-1])
print(latest_file)

model.load_weights(latest_file)

test_score = model.evaluate(X_val, y_val)
predictions = model.predict(X_val)
predicted_classes = np.argmax(predictions, axis=1)
precision1, recall1, f11, _ = precision_recall_fscore_support(y_val, predicted_classes, average='weighted')
print(test_score[1])


Epoch 1/3
21/21 [==============================] - ETA: 0s - loss: 0.2795 - accuracy: 0.9048
Epoch 1: val_loss improved from inf to 0.41416, saving model to saved_models\weights_improvement-01-0.4142.hdf5
21/21 [==============================] - 50s 2s/step - loss: 0.2795 - accuracy: 0.9048 - val_loss: 0.4142 - val_accuracy: 0.8562
Epoch 2/3
21/21 [==============================] - ETA: 0s - loss: 0.2196 - accuracy: 0.9315
Epoch 2: val_loss improved from 0.41416 to 0.37112, saving model to saved_models\weights_improvement-02-0.3711.hdf5
21/21 [==============================] - 45s 2s/step - loss: 0.2196 - accuracy: 0.9315 - val_loss: 0.3711 - val_accuracy: 0.8687
Epoch 3/3
21/21 [==============================] - ETA: 0s - loss: 0.1804 - accuracy: 0.9494
Epoch 3: val_loss improved from 0.37112 to 0.34203, saving model to saved_models\weights_improvement-03-0.3420.hdf5
21/21 [==============================] - 45s 2s/step - loss: 0.1804 - accuracy: 0.9494 - val_loss: 0.3420 - val_accurac

In [55]:
model.load_weights('saved_models\\weights_improvement-02-0.3697.hdf5')

In [57]:
import os

directory = "saved_models"  # Replace with the actual directory path

for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    print(file_path)
    if os.path.isfile(file_path):
        os.remove(file_path)
    print(f'{file_path} is deleted\n')

saved_models\weights_improvement-01-0.3889.hdf5
saved_models\weights_improvement-01-0.3889.hdf5 is deleted

saved_models\weights_improvement-01-0.4142.hdf5
saved_models\weights_improvement-01-0.4142.hdf5 is deleted

saved_models\weights_improvement-02-0.3697.hdf5
saved_models\weights_improvement-02-0.3697.hdf5 is deleted

saved_models\weights_improvement-02-0.3711.hdf5
saved_models\weights_improvement-02-0.3711.hdf5 is deleted

saved_models\weights_improvement-03-0.3420.hdf5
saved_models\weights_improvement-03-0.3420.hdf5 is deleted

